In [ ]:
import os
import glob
import json
from collections import Counter, defaultdict

import pandas as pd

# ======================
# 0. 경로 설정
# ======================
root    = "/Users/youuchul/Documents/github/03_projects/01_HealthEat Pill Detection Model/toss_files/251210_어노테이션통합_자동라벨링/unified_dataset"
ann_dir = os.path.join(root, "annotations")

# ======================
# 1. 어노테이션 파일 목록 수집
# ======================
ann_paths = glob.glob(os.path.join(ann_dir, "*.json"))
print("어노테이션 JSON 파일 수:", len(ann_paths))

# ======================
# 2. 클래스 통계 집계용 객체
# ======================

# 클래스별 박스 개수
cat_box_count = Counter()

# 클래스별 등장 이미지 수 (한 이미지에 여러 박스 있어도 1로 카운트)
cat_image_count = Counter()

# id -> name / supercategory 매핑
cat_name_map = {}
cat_super_map = {}

# ======================
# 3. JSON 순회하면서 집계
# ======================

for json_path in ann_paths:
    with open(json_path, "r") as f:
        data = json.load(f)

    # 3-1) 카테고리 정보 갱신 (id -> name, supercategory)
    for c in data.get("categories", []):
        cid  = c["id"]
        name = c.get("name", "")
        supercat = c.get("supercategory", "")

        # 이미 저장된 이름/슈퍼카테고리와 다르면 체크(이상한 경우 디버깅용)
        if cid in cat_name_map and cat_name_map[cid] != name:
            print(f"[WARN] category_id {cid} 이름 불일치: '{cat_name_map[cid]}' vs '{name}' (file={os.path.basename(json_path)})")
        else:
            cat_name_map[cid] = name

        if cid in cat_super_map and cat_super_map[cid] != supercat:
            print(f"[WARN] category_id {cid} supercategory 불일치: '{cat_super_map[cid]}' vs '{supercat}' (file={os.path.basename(json_path)})")
        else:
            cat_super_map[cid] = supercat

    # 3-2) 어노테이션(박스) 정보 집계
    anns = data.get("annotations", [])

    # 이 이미지에 등장한 class id 모음 (이미지 수 카운트용)
    cats_in_this_image = set()

    for ann in anns:
        cid = ann["category_id"]
        cat_box_count[cid] += 1
        cats_in_this_image.add(cid)

    # 이미지당 1회만 추가
    for cid in cats_in_this_image:
        cat_image_count[cid] += 1

# ======================
# 4. 집계 결과 → DataFrame
# ======================

rows = []
for cid in sorted(cat_box_count.keys()):
    rows.append({
        "category_id": cid,
        "category_name": cat_name_map.get(cid, ""),
        "supercategory": cat_super_map.get(cid, ""),
        "num_boxes": cat_box_count[cid],       # 전체 박스 수
        "num_images": cat_image_count[cid],    # 등장한 이미지 수
    })

df_cat_summary = pd.DataFrame(rows)
print("클래스 요약 df_cat_summary shape:", df_cat_summary.shape)
print(df_cat_summary.head())

# ======================
# 5. CSV로 저장
# ======================

output_dir = "./eda_outputs"
os.makedirs(output_dir, exist_ok=True)

csv_cat_path = os.path.join(output_dir, "pill_category_summary.csv")
df_cat_summary.to_csv(csv_cat_path, index=False, encoding="utf-8-sig")

print(f"클래스 요약 CSV 저장 완료 → {csv_cat_path}")